In [1]:
# agent/graph.py

from __future__ import annotations

import json
import unicodedata
from typing import TypedDict, List, Optional, Literal, Dict, Any

from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

from rapidfuzz import process, fuzz


# -----------------------------------------------------------------------------
# 0. GraphState 정의 (state.py에 별도 분리해도 됨)
# -----------------------------------------------------------------------------

class GraphState(TypedDict, total=False):
    messages: List[BaseMessage]
    question: str
    kg_result: Optional[Dict[str, Any]]
    rag_docs: Optional[List[Any]]
    answer: Optional[str]
    route: Optional[Literal["kg", "rag"]]


# -----------------------------------------------------------------------------
# 1. Env / 전역 리소스 초기화 (벡터스토어, KG, LLM, retriever, prompt)
# -----------------------------------------------------------------------------

load_dotenv()

# 🔹 벡터스토어
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings,
    collection_name="english_grammar_chunked",
)

# 🔹 Knowledge Graph
with open("/data/edutem/sooine/rag_bot/knowledge_graph.json", "r", encoding="utf-8") as f:
    KNOWLEDGE_GRAPH = json.load(f)

# 🔹 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 🔹 Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 친절한 영어 학습 도우미입니다. 아래 context를 바탕으로 답변하세요.\n\n<context>\n{context}\n</context>",
        ),
        MessagesPlaceholder("chat_history"),
        ("human", "{question}"),
    ]
)

# 🔹 고급 Retriever (Contextual Compression)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 15})
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor,
)

# -----------------------------------------------------------------------------
# 2. KG 유틸 (네 코드 그대로 옮김)
# -----------------------------------------------------------------------------

def normalize(s: str):
    return unicodedata.normalize("NFC", s.lower().replace(" ", ""))


def fuzzy_match_topic(query, topic_list):
    q = normalize(query)
    candidates = [normalize(t) for t in topic_list]
    match, score, idx = process.extractOne(q, candidates, scorer=fuzz.ratio)
    return topic_list[idx] if score > 70 else None


def search_in_knowledge_graph(query: str) -> Optional[Dict[str, Any]]:
    q = query.lower().strip()
    topic_list = list(KNOWLEDGE_GRAPH.keys())

    best = fuzzy_match_topic(query, topic_list)
    if best:
        return {
            "type": "main_topic",
            "main_topic": best,
            "data": KNOWLEDGE_GRAPH[best],
        }

    for topic in topic_list:
        if topic.lower() == q:
            return {
                "type": "main_topic",
                "main_topic": topic,
                "data": KNOWLEDGE_GRAPH[topic],
            }

    for topic in topic_list:
        if q in topic.lower():
            return {
                "type": "main_topic",
                "main_topic": topic,
                "data": KNOWLEDGE_GRAPH[topic],
            }

    best_match = None
    best_score = 0
    for main_topic, topic_data in KNOWLEDGE_GRAPH.items():
        for sub_id, sub_data in topic_data["sub_topics"].items():
            score = 0
            if q in sub_data["title"].lower():
                score += 3
            if q in sub_data["concept"].lower():
                score += 2
            for ex in sub_data.get("examples", []):
                if q in ex.lower():
                    score += 1
                    break
            if score > best_score:
                best_score = score
                best_match = {
                    "type": "sub_topic",
                    "main_topic": main_topic,
                    "sub_topic_id": sub_id,
                    "data": sub_data,
                }

    if best_match and best_score >= 1:
        return best_match

    return None


def format_kg_result_for_answer(kg_result: Dict[str, Any]) -> str:
    """Streamlit에서 하던 출력 패턴을 텍스트로 합치기."""
    data = kg_result["data"]
    concept = data.get("concept", "")
    examples = data.get("examples", [])

    lines = [concept]
    if examples:
        lines.append("\n예문:")
        for ex in examples:
            lines.append(f"- {ex}")
    return "\n".join(lines)


# -----------------------------------------------------------------------------
# 3. RAG 유틸
# -----------------------------------------------------------------------------

def format_docs(docs):
    return "\n\n".join(getattr(doc, "page_content", "") for doc in docs)


# -----------------------------------------------------------------------------
# 4. Nodes
# -----------------------------------------------------------------------------

def prepare_question_node(state: GraphState) -> Dict:
    """messages에서 마지막 HumanMessage를 question으로 정규화."""
    if state.get("question"):
        return {}
    msgs = state.get("messages", [])
    q = ""
    for m in reversed(msgs):
        if isinstance(m, HumanMessage):
            q = m.content
            break
    return {"question": q}


def search_kg_node(state: GraphState) -> Dict:
    """KG에서 먼저 찾고, 있으면 route='kg', 없으면 route='rag'."""
    q = state.get("question", "")
    kg_result = search_in_knowledge_graph(q)

    if kg_result:
        return {
            "kg_result": kg_result,
            "route": "kg",
        }
    else:
        return {
            "kg_result": None,
            "route": "rag",
        }


def answer_from_kg_node(state: GraphState) -> Dict:
    """KG hit인 경우: KG 데이터로 바로 answer 생성."""
    msgs = state.get("messages", [])
    kg_result = state.get("kg_result")
    if not kg_result:
        ai = AIMessage(content="지식 그래프에서 관련 내용을 찾지 못했습니다.")
        return {"answer": ai.content, "messages": msgs + [ai]}

    text = format_kg_result_for_answer(kg_result)
    ai = AIMessage(content=text)
    return {"answer": text, "messages": msgs + [ai]}


def retrieve_rag_node(state: GraphState) -> Dict:
    """KG에 없으면: 고급 retriever로 문서 검색."""
    q = state.get("question", "")
    docs = compression_retriever.invoke(q)
    return {"rag_docs": docs}


def generate_rag_answer_node(state: GraphState) -> Dict:
    """retriever 결과 + prompt + LLM → answer."""
    msgs = state.get("messages", [])
    q = state.get("question", "")
    docs = state.get("rag_docs") or []

    context = format_docs(docs)

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke(
        {
            "context": context,
            "question": q,
            "chat_history": msgs,  # history를 prompt에 그대로 넣어도 됨
        }
    )

    ai = AIMessage(content=answer)
    return {"answer": answer, "messages": msgs + [ai]}


# -----------------------------------------------------------------------------
# 5. 조건 분기
# -----------------------------------------------------------------------------

def route_after_kg(state: GraphState) -> str:
    """search_kg_node 이후 어디로 갈지 결정."""
    route = state.get("route")
    if route == "kg":
        return "kg"
    return "rag"


# -----------------------------------------------------------------------------
# 6. 그래프 구성
# -----------------------------------------------------------------------------

def build_graph():
    g = StateGraph(GraphState)

    g.add_node("prepare_question", prepare_question_node)
    g.add_node("search_kg", search_kg_node)
    g.add_node("answer_from_kg", answer_from_kg_node)
    g.add_node("retrieve_rag", retrieve_rag_node)
    g.add_node("generate_rag_answer", generate_rag_answer_node)

    g.set_entry_point("prepare_question")

    g.add_edge("prepare_question", "search_kg")

    # KG hit 여부에 따라 분기
    g.add_conditional_edges(
        "search_kg",
        route_after_kg,
        {
            "kg": "answer_from_kg",
            "rag": "retrieve_rag",
        },
    )

    # RAG path
    g.add_edge("retrieve_rag", "generate_rag_answer")
    g.add_edge("answer_from_kg", END)
    g.add_edge("generate_rag_answer", END)

    app = g.compile(checkpointer=MemorySaver())
    return app


graph = build_graph()


if __name__ == "__main__":
    internal = graph.get_graph()
    print(internal.draw_mermaid())
    internal.draw_mermaid_png("kg_rag_pipeline.png")



/data/edutem/.cache/pypoetry/virtualenvs/rag-bot-vbdTYmCJ-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain_community.retrievers.contextual_compression'

In [ ]:

if __name__ == "__main__":
    internal = graph.get_graph()
    print(internal.draw_mermaid())
    internal.draw_mermaid_png("video_rag_detailed_pipeline.png")